In [40]:
import json
from enum import Enum
from pathlib import Path
import psycopg2
import ollama
import pandas as pd
from IPython.display import Image, Markdown, display
from tqdm import tqdm
import os
from dotenv import load_dotenv, find_dotenv
import urllib.parse
from langchain.messages import AIMessage
from langchain_ollama import ChatOllama
from langchain_deepseek import ChatDeepSeek
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_agent
from agentic_ai_tools import PaymentReferenceSearch, AccessAccountsReceivable, AccessPayments,DataFramePayload
import datetime as dt
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy
from langchain_experimental.agents import create_pandas_dataframe_agent
from typing import Union, List, Dict
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

In [41]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional

class ReconciliationState(TypedDict):
    ar: Optional[DataFramePayload]
    pr: Optional[DataFramePayload]
    result: Optional[DataFramePayload]
    review: str


In [42]:
# Langchain API KEY
LANGSMITH_API_KEY=os.getenv('LANGSMITH_API_KEY')
LANGSMITH_ENDPOINT=os.getenv('LANGSMITH_ENDPOINT')
# DATABASE Connection settings
DB_NAME=os.getenv('DB_NAME')
USERNAME=os.getenv('USERNAME')
PASSWORD=urllib.parse.quote(os.getenv('PASSWORD'))
HOSTNAME=os.getenv('HOSTNAME')
PORT=os.getenv('PORT')

# Creating conneciton to database
conn = psycopg2.connect(f"dbname={DB_NAME} user={USERNAME} password={PASSWORD}")
accounts_receivables = pd.read_sql("SELECT * FROM accounts_receivable", conn)
payments= pd.read_sql("SELECT * FROM payments", conn)

/var/folders/4s/q4zndhns13bdmf7q1_8p3crc0000gn/T/ipykernel_21569/3319540732.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  accounts_receivables = pd.read_sql("SELECT * FROM accounts_receivable", conn)
/var/folders/4s/q4zndhns13bdmf7q1_8p3crc0000gn/T/ipykernel_21569/3319540732.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  payments= pd.read_sql("SELECT * FROM payments", conn)


In [43]:
class Context():
    """Custom runtime context schema."""
    user_id: str

In [44]:
prompt =f"""You are an expert accountant in the accounts receivables department. You have access to the following tools:
            - AccessAccountsReceivable to retrieve the current accounts receivables data.
            - AccessPayments to retrieve the payments received.
            - PaymentReferenceSearch to match the payments received to the accounts receivables, returning an updated accounts receivable dataframe and and updated payments dataframe.
            You need to AccessAccountsReceivable and AccessPayments first, before you access PaymentReferenceSearch.
            If a user asks you to show the accounts receivables, payments data, or to update accounts receivables, make sure you use these tools.
            Be concise and accurate.
            """

In [45]:
# Add memory to your agent to maintain state across interactions. This allows the agent to remember previous conversations and context.
checkpointer = InMemorySaver()

In [46]:
model = ChatOllama(
    model="gpt-oss:20b",
    temperature=0,
) 

In [47]:
def load_ar(state: ReconciliationState):
    """First call to retrieve Accounts Receivable"""
    ar = AccessAccountsReceivable.invoke({})
    return {"ar": ar}


In [48]:
def load_pr(state: ReconciliationState):
    """Call to retrieve Payments Received"""
    pr = AccessPayments.invoke({})
    return {"pr": pr}


In [49]:
def reconcile(state: ReconciliationState):
    """Call to retrieve reconciliation"""
    result = PaymentReferenceSearch.invoke({
        "ar": state["ar"],
        "pr": state["pr"]
    })
    return {"result": result}


In [50]:
def review(state:ReconciliationState):
    """LLM Call to summarise the data from the reconciliation and list unmatched invoices and payments"""
    msg = model.invoke(f"Summarise how many invoices are still open from: {ReconciliationState['result']}, which is a DataFrame Payload of the updated accounts receivables table.")
    return {"review":msg.content}

In [51]:
workflow = StateGraph(ReconciliationState)

workflow.add_node("load_ar", load_ar)
workflow.add_node("load_pr", load_pr)
workflow.add_node("reconcile", reconcile)
workflow.add_node("review", review)

workflow.set_entry_point("load_ar")
workflow.add_edge("load_ar", "load_pr")
workflow.add_edge("load_pr", "reconcile")
workflow.add_edge("reconcile", "review")
workflow.add_edge("review",END)

graph = workflow.compile()


In [52]:
result_state = graph.invoke({})
final_df = result_state["result"].to_df()

/Users/davidal-gurnawi/Documents/data_science/agentic_test/agentic_ai_tools.py:143: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  accounts_receivables = pd.read_sql("SELECT * FROM accounts_receivable", conn)
/Users/davidal-gurnawi/Documents/data_science/agentic_test/agentic_ai_tools.py:149: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  payments = pd.read_sql("SELECT * FROM payments", conn)


12038 - 1 - We'll pay the rest later
['12038', '-', '1', '-', "We'll", 'pay', 'the', 'rest', 'later']
Element: 12038
Starting Customer Number Match
Customer Match!
Element: -
Starting Invoice Number Match
Error Invoice Match: cannot unpack non-iterable NoneType object
Element: 1
Starting Invoice Number Match
Invoice Number Match!
Invoice and Customer matched!
Customer Nr 12000 - Invoice Nr 2 - Mom's
['Customer', 'Nr', '12000', '-', 'Invoice', 'Nr', '2', '-', "Mom's"]
Element: Customer
Starting Customer Number Match
Error Customer Match: cannot unpack non-iterable NoneType object
Element: Nr
Starting Customer Number Match
Error Customer Match: cannot unpack non-iterable NoneType object
Element: 12000
Starting Customer Number Match
Customer Match!
Element: -
Starting Invoice Number Match
Error Invoice Match: cannot unpack non-iterable NoneType object
Element: Invoice
Starting Invoice Number Match
Error Invoice Match: cannot unpack non-iterable NoneType object
Element: Nr
Starting Invoice